# Importing Packages

In [9]:
#the basics...
import numpy as np
import pandas as pd

#used for moving files around and finding folders
import os as os

#used to output pretty progress bars
from tqdm import tqdm

import gc

In [10]:
#suppress warning from pandas
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)

In [17]:
#ensuring certain directories are present;
newpath1 = r'RETA_data'
newpath2 = r'output' 
if not os.path.exists(newpath1):
    os.makedirs(newpath1)

if not os.path.exists(newpath2):
    os.makedirs(newpath2)

# Defining the Function

In [16]:
def read_RETA_file(filepath, keyfile = 'RETURN-A_Keyfile.xlsx', crosswalk_path = None):
    
    #first, read the Key File
    key_file = pd.read_excel(keyfile, sheet_name = "New Variables")
    
    #getting a shortened 3-letter month string
    key_file['month_short'] = key_file['month'].str.lower().str[:3]

    #combining each column name with its month
    key_file['name'] = key_file['month_short'] + '||' + key_file['new_names_for_real_this_time']
    
    
    #reading the RETA file as a list of each line in the file;
    with open(filepath, 'r') as file:
        orig_txt = file.readlines()
    
    #doing a tiny bit of cleanup;
    #  the \n was retained at the end of each line, so i am removing that
    for i in range(len(orig_txt)):
        orig_txt[i] = orig_txt[i].replace('\n','')

    #preparing the dataframe to parse the RETURN-A file into
    wide_df = pd.DataFrame()

    #creating a string column, where each row contains the entire line from the RETURN-A file.
    wide_df['origtxt'] = orig_txt
    
    #iterate through the different columns from the keyfile.
    curr_pos = 0 #tracking the position along the lines.
    for i in tqdm(range(len(key_file)), 'Collecting Column Information'):

        #identify the datatype;
        dtype_ind = key_file['Type_Length'].iloc[i][0] #"N is numeric, A is character"

        #identify how long this entry is.
        entry_length = int(key_file['Type_Length'].iloc[i][1:])

        #getting the df;
        #  this uses the pandas string accessor;
        #  makes a new column that is a subset of the 'origtxt' column
        #  names it based on the current column name
        wide_df[key_file['name'].iloc[i]] = wide_df['origtxt'].str[curr_pos:curr_pos + entry_length]

        #converts the column to numeric if they are marked as such
        if(dtype_ind == 'N'):
            wide_df[key_file['name'].iloc[i]] = pd.to_numeric(wide_df[key_file['name'].iloc[i]], errors = 'coerce')

        #updating the current position.
        curr_pos += entry_length
    
    
    #dropping the original text column;
    wide_df.drop(columns = ['origtxt'], inplace = True)
    
    #trying to aggregate this stuff;
    
    #first, get substrings corresponding to the relevant columns
    #   gets the last 31 columns in the list
    #   this corresponds to the columns for december.
    name_bits = pd.Series(key_file['name'][-31:]).copy()

    #removing the 'dec||' prefix from the column names that have it, and adding '_2' to these
    #   _2 corresponds to the actual offenses.
    name_bits.iloc[:-3] = name_bits.iloc[:-3].str[5:-2] + '_2'

    #removing the 'dec||' from the column names that have it
    name_bits.iloc[-3:] = name_bits.iloc[-3:].str[5:]

    #making a second array of names that correspond to the other columns that we want to drop.
    drop_bits = pd.concat([name_bits.copy().iloc[:-3].str[:-1] + '1',
                           name_bits.copy().iloc[:-3].str[:-1] + '3',
                           name_bits.copy().iloc[:-3].str[:-1] + '4'])
    
    #converting these from pandas Series to arrays of strings.
    name_bits = np.array(name_bits)
    drop_bits = np.array(drop_bits)
    
    #for each;
    #  find what rows contain that substring
    #  add all of those rows together into a new column
    #  drop those old columns from the overall dataframe.
    for i in tqdm(range(len(name_bits)), 'Aggregating Column Information'):

        #find what rows of the dataframe contain this 'name_bit' (something like murder_2)
        cols_mask = np.zeros(len(wide_df.columns), dtype = bool)
        for j in range(len(cols_mask)):
            cols_mask[j] = name_bits[i] in wide_df.columns[j]

        #making a dataframe that only contains the relevant columns
        cols_of_interest = wide_df.columns[cols_mask]

        #converting each of those columns to numeric;
        for j in range(len(cols_of_interest)):
            wide_df[cols_of_interest[j]] = pd.to_numeric(wide_df[cols_of_interest[j]], errors='coerce')

        #now, we want to add all those columns together
        wide_df[name_bits[i]] = wide_df[cols_of_interest].sum(axis = 1, skipna=True)

        #dropping these aggregated columns from the dataframe.
        wide_df.drop(columns = cols_of_interest, inplace = True)

        
    #dropping extra columns that correspond to other kinds of counts (unfounded arrests, cleared by arrests, arrests under 18)
    for i in range(len(drop_bits)):
        
        #find what rows of the dataframe contain this 'drop_bit' (something like 'murder_3', 'robbery_1')
        cols_mask = np.zeros(len(wide_df.columns), dtype = bool)
        for j in range(len(cols_mask)):
            cols_mask[j] = drop_bits[i] in wide_df.columns[j]

        #dropping these identified columns.
        wide_df.drop(columns = wide_df.columns[cols_mask], inplace = True)
        
    
    #dropping a set of columns that are otherwise of no use to us.
    #   they represent information about the columns we just dropped.
    
    #identify these columns.
    cols_mask = np.zeros(len(wide_df.columns), dtype = bool)
    for j in range(len(cols_mask)):
        if('card0' in wide_df.columns[j]):
            cols_mask[j] = True

        if('card2' in wide_df.columns[j]):
            cols_mask[j] = True

        if('card3' in wide_df.columns[j]):
            cols_mask[j] = True

    #dropping those columns.
    wide_df.drop(columns = wide_df.columns[cols_mask], inplace = True)
    
    #creating a new months_reported column based on the card1_type column for each month;
    
    #identifying the columns
    cols_mask = np.zeros(len(wide_df.columns), dtype = bool)
    for j in range(len(cols_mask)):
        if('card1_type' in wide_df.columns[j]):
            cols_mask[j] = True

    #paring to just these columns
    card1_cols = wide_df.columns[cols_mask]

    #preparing the new_months_reported column
    wide_df['new_months_reported'] = np.zeros(len(wide_df))
    
    #for each column (month)
    for i in range(len(card1_cols)):
        
        #get a column of the numbers as integers.
        temp = pd.to_numeric(wide_df[card1_cols[i]], errors = 'coerce')

        #get a column of true/false, representing if that month has data
        col_bool = (temp == 5) | (temp == 2)

        #convert it to an array
        col_bool = np.array(col_bool)
        
        #this is the tricky part;
        #   when you convert a boolean array to an integer, it becomes either 0 (false) or 1 (true)
        #   here, I am adding that column to the new_months_reported; true = +1, false = +0.
        wide_df['new_months_reported'] = np.array(wide_df['new_months_reported']) + col_bool.astype(int)
    

    if(crosswalk_path != None):

        #reading the crosswalk file.
        crosswalk = pd.read_csv(crosswalk_path, sep = '	', dtype = 'string')

        #paring to what we want;
        crosswalk = crosswalk.loc[crosswalk.ORI7 != '-1',['ORI7', 'FIPS_ST','FIPS_COUNTY', 'FPLACE']]

        #renaming in preparation to merge;
        crosswalk.rename(columns = {'ORI7' : 'hea||ori',
                                    'FIPS_ST' : 'STATEFP',
                                    'FIPS_COUNTY' : 'COUNTYFP',
                                    'FPLACE' : 'PLACEFP'}, inplace = True)
        #mergin;
        wide_df = wide_df.merge(crosswalk, on='hea||ori', how='left')

    return(wide_df)

# Running
In order for this file to run, you will need to download the RETURN-A data from https://cde.ucr.cjis.gov/LATEST/webapp/#, under Documents & Downloads, then Master File Downloads.

Once downloaded, they must be placed in the 'RETA_data' folder. They must be renamed, such that the year of the data is at the front, separated by a '_'.
(Ex: KCRETA85.DAT --> 1985_KCRETA85.DAT).



In [14]:
#if you have downloaded the ICPSR crosswalk file, which is used to associate state, county, and place fips codes, specify the path here;
#   the ICPSR crosswalk file can be found at: https://www.icpsr.umich.edu/web/ICPSR/studies/35158#
crosswalk_path = None

In [15]:
#Listing the identified RETURN-A files.
orig_RETA_files = os.listdir('RETA_data')
print(orig_RETA_files)

['1985_KCRETA85.DAT', '1986_KCRETA86.DAT', '1987_KCRETA87.DAT', '1988_KCRETA88.DAT', '1989_KCRETA89.DAT', '1990_KEN90', '1991_KEN91', '1992_KEN92', '1993_KEN93', '1994_KEN94', '1995_KEN95', '1996_KEN96', '1997_RETAFIX.Y97', '1998_KCRETA98.DAT', '1999_KCRETA99.DAT', '2000_KCRETA00.DAT', '2001_RETA01.y01', '2002_RETA02', '2003_RETA03.DAT', '2004_RETA04.DAT', '2005_RETA05.DAT', '2006_RETA06.DAT', '2007_RETA07.DAT', '2008_RETA08.DAT', '2009_RETA09.DAT', '2010_RETA10.DAT', '2011_RETA11.DAT', '2012_RETA12.DAT', '2013_RETURNA2013.TXT', '2014_RETA14.DAT', '2015_RETA-COMB.txt', '2016_RETA2016.TXT', '2017_RETA_NATIONAL_MASTER_FILE.txt', '2018_reta-2018.txt', '2019_RETA_NATIONAL_MASTER_FILE_STATIC.txt', '2020_RETA_NATIONAL_MASTER_FILE.txt', '2021_RETA_NATIONAL_MASTER_FILE.txt', '2022_RETA_NATIONAL_MASTER_FILE.txt']


In [13]:
#first, read the files in the folder;
orig_RETA_files = os.listdir('RETA_data')

#for each identified file;
for i in range(len(orig_RETA_files)):
    
    #get this specific filename
    RETA_file = orig_RETA_files[i]
    
    #get the year from the filename;
    year = int(RETA_file[:4])
    
    print('==================  %s  ======================='%year)
    print('filename: %s'%RETA_file)
    
    #using the function
    output_df = read_RETA_file('RETA_data/' + RETA_file, crosswalk_path=crosswalk_path)
    
    #setting the year to the actual, rather than two digits
    output_df['hea||year'] = year
    
    #saving;
    output_df.to_csv('output/' + 'RETA' + str(year) + '.csv', index = False)
    

==================  2017  =======================
filename: 2017_RETA_NATIONAL_MASTER_FILE.txt


Aggregating Column Information: 100%|██████████| 31/31 [00:19<00:00,  1.58it/s]
